# Single sample analysis

## Parsing the outputs with answers (BioData1)

In [413]:
f_ans = open("./Answer.txt", 'r')
dic_ans = {"TRUE":[], 'FALSE':[], "TRUE_VAF" : [], "FALSE_VAF": []}

#For set analysis
l_TP_set = []
l_FP_set = []


for line in f_ans:
    #print (line)
    s = line.strip().split('\t')
    if line[0] == "#":
        l_header = s
        print (l_header)
    else:
        dic = dict(zip(l_header, s))
        #print (dic)
        sample = dic["Sample"]
        chrom = dic["#CHROM"]
        pos = dic["POS"]
        ref = dic["REF"]
        alt = dic["ALT"]
        VAF = float(dic['VAF_Validation'].rstrip('%')) * 0.01
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        shared_stat = None

        validation = dic["Validation"]
        if validation == "TRUE":
            dic_ans['TRUE'].append(info)
            dic_ans['TRUE_VAF'].append(float(VAF))
            l_TP_set.append(info)
        else:
            dic_ans['FALSE'].append(info)
            dic_ans['FALSE_VAF'].append(float(VAF))
            l_FP_set.append(info)

for stat in dic_ans:
    print (stat, len(dic_ans[stat]))
    

print (dic_ans["TRUE"][0])
#def Performance_gather(info, VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF):
def Performance_gather(info, VAF, cnt_true, cnt_false,l_false, l_false_VAF):
    if info in dic_ans["TRUE"]:
            cnt_true +=1
            l_true.append(info)
    elif info in dic_ans["FALSE"]:
            cnt_false +=1
            l_false.append(info)
            l_false_VAF.append(VAF)
    else:
        pass
            #print (info, "Neither in TRUE nor FALSE")
    return [cnt_true, cnt_false, l_false, l_false_VAF]

from scipy.stats import hmean
def Performance_check(cnt_true, cnt_false):
    TP = cnt_true
    FP = cnt_false
    PC = len(dic_ans['TRUE'])
    NC = len(dic_ans['FALSE'])
    sens = TP/PC
    if TP == 0 and FP== 0:
        pre = None
        f_score = None
    else:
        pre = TP/(TP + FP)
        f_score = hmean([sens,pre])
    print (sens, pre, f_score)
    return (TP, FP, sens, pre, f_score)


['#CHROM', 'POS', 'REF', 'ALT', ' Mappability ', 'VAF_Validation', 'Sample', 'Group', 'Validation']
TRUE 117
FALSE 13
TRUE_VAF 117
FALSE_VAF 13
GBM-P-0396-Peri-M-Rep1:chr1:10659715:C:T


In [401]:
DATA = "Kim et al."
VAF_range = "Total"
l_result = []
out = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_result.txt', 'w')
out.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" %("tool", "TP", "FP", "Sensitivity", "Precision", "F1-score", "Data", "VAF_range"))
out_set_TP = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_set_TP.txt', 'w')
out_set_FP = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_set_FP.txt', 'w')

In [402]:
#MF
f = open('/data/project/MRS/BioValidation/PlosGen/output/MF/All.snv.Predictions.Mosaic', 'r')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0

for line in f:
    #print (line)
    s = line.strip().split('\t')
    if s[0] == "id":
        l_header = s
    else:
        dic = dict(zip(l_header, s))
        info_tmp = dic['id']
        VAF = dic['AF']
        info_tmp = info_tmp.split('/')[-1]
        #print (info_tmp.split('~'))
        info = ':'.join(info_tmp.split('~'))
        #print (info) #AD-P-2013029-HIF-F-Rep1:chr8:123182166:A:G
        cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
        cnt_true = cnts[0]
        cnt_false = cnts[1]
        l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_true)
print (l_false)
print (l_false_VAF)



l_result_tmp = ['MF']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')

l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
        
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("MF" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("MF" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_false)
print (l_FP_decide)

0.6324786324786325 0.9367088607594937 0.7551020408163265
(74, 5, 0.6324786324786325, 0.9367088607594937, 0.7551020408163265)
['AD-P-2013029-HIF-F-Rep1:chr8:123182166:A:G', 'AD-P-2013029-HIF-F-Rep1:chr22:40315361:G:A', 'AD-P-2013066-HIF-F-Rep1:chr4:99347143:A:G', 'AD-P-2013066-HIF-F-Rep1:chr13:42992009:G:A', 'AD-P-2013066-HIF-F-Rep1:chr15:98896836:A:T', 'AD-P-2013066-HIF-F-Rep1:chr20:17494172:C:T', 'AD-P-2013066-HIF-F-Rep1:chr22:46257097:G:A', 'AD-P-2014054-HIF-F-Rep1:chr11:67628329:C:T', 'ASD-P-1349-Occi-M-Rep1:chr12:4626950:G:T', 'ASD-P-1638-PFC-F-Rep1:chr12:113191328:G:A', 'ASD-P-4899-Occi-M-Rep1:chr5:179559872:T:A', 'ASD-P-5574-Heart-M-Rep1:chr15:74722831:C:T', 'ASD-P-5574-Occi-M-Rep1:chr9:84294263:C:T', 'ASD-P-5574-Occi-M-Rep1:chr15:74722831:C:T', 'FCD-P-0021-Blood-M-Rep1:chr16:69463844:A:G', 'FCD-P-0021-RTemOcci-M-Rep1:chr16:69463844:A:G', 'FCD-P-0023-Blood-M-Rep1:chr20:36209646:G:A', 'FCD-P-0023-Lhem-M-Rep1:chr20:36209646:G:A', 'GBM-P-0026-Tumor-F-Rep2:chr11:128480240:G:A', 'GBM-

In [403]:
#MT2
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/MT2/*PASS.vcf")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            print (s[8])
            VAF = s[9].split(':')[2]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['MT2']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')

l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("MT2" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("MT2" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_TP_decide)

GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1

In [404]:
#DM
f = open('/data/project/MRS/BioValidation/PlosGen/output/DM_output/All.DM.output.mosaic.txt', 'r')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0

for line in f:
    #print (line)
    s = line.strip().split('\t')
    sample = s[0]
    chrom = s[2]
    pos = s[3]
    ref = s[4]
    alt = s[5]
    VAF = s[7]
    info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt

    cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
    cnt_true = cnts[0]
    cnt_false = cnts[1]
    l_false = cnts[2]
    
print (cnt_true, cnt_false)
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['DM']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')

l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("DM" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("DM" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_TP_decide)

15 2
0.1282051282051282 0.8823529411764706 0.22388059701492538
(15, 2, 0.1282051282051282, 0.8823529411764706, 0.22388059701492538)
['SCZ-C-1014-PFC-M-Rep2:chr2:138015377:A:G', 'SCZ-P-1001-PFC-M-Rep1:chr13:112558343:G:A']
['0.048', '0.02564102564102564']
0.1282051282051282 0.8823529411764706 0.22388059701492538
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0']


In [405]:
#HCp20
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/HCp20/*PASS.vcf")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['HC-p20']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')

l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("HC-p20" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("HC-p20" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_TP_decide)

0.3418803418803419 0.9523809523809523 0.5031446540880503
(40, 2, 0.3418803418803419, 0.9523809523809523, 0.5031446540880503)
['NLE-P-0228-Occi-F-Rep1:chr7:100964707:C:T', 'GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A']
[0.245662100456621, 0.27673649393605293]
0.3418803418803419 0.9523809523809523 0.5031446540880503
['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1']


In [406]:
#HCp200
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/HCp200/*PASS.vcf")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['HC-p200']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("HC-p200" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("HC-p200" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_TP_decide)

0.38461538461538464 0.9375 0.5454545454545455
(45, 3, 0.38461538461538464, 0.9375, 0.5454545454545455)
['GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T', 'NLE-P-0228-Occi-F-Rep1:chr7:100964707:C:T', 'GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A']
[0.08284023668639054, 0.245662100456621, 0.27673649393605293]
0.38461538461538464 0.9375 0.5454545454545455
['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1']


In [407]:
#MH
import glob
fl = glob.glob('/data/project/MRS/BioValidation/PlosGen/output/MH/*/run/final.passed.tsv')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    sample = str(file).split('/')[-3]
    for line in f:
        s = line.strip().split('\t')
        chrom = s[0]
        pos = s[1]
        ref = s[6]
        alt = s[8]
        VAF = float(s[9])/int(s[7])
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
        cnt_true = cnts[0]
        cnt_false = cnts[1]
        l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)
        
l_result_tmp = ['MH']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp )) + '\t'+ DATA + '\t' + VAF_range + '\n')


l_TP_decide = []
l_FP_decide = []
for locus in l_TP_set:
    if locus in l_true:
        l_TP_decide.append("1")
    else:
        l_TP_decide.append("0")
for locus in l_FP_set:
    if locus in l_false:
        l_FP_decide.append("1")
    else:
        l_FP_decide.append("0")

out_set_TP.write("MH" + '\t' + '\t'.join(l_TP_decide) + '\n')
out_set_FP.write("MH" + '\t' + '\t'.join(l_FP_decide) + '\n')

print (l_TP_decide)

0.1111111111111111 0.8666666666666667 0.196969696969697
(13, 2, 0.1111111111111111, 0.8666666666666667, 0.196969696969697)
['NLE-P-0228-Occi-F-Rep1:chr7:100964707:C:T', 'SCZ-C-1014-PFC-M-Rep2:chr2:138015377:A:G']
[0.32675709001233044, 0.05309734513274336]
0.1111111111111111 0.8666666666666667 0.196969696969697
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


## Feature-level recombination

In [3]:
f_ans = open("../../resource/Answer.txt", 'r')
dic_ans = {"TRUE":[], 'FALSE':[], "TRUE_VAF" : [], "FALSE_VAF": []}
for line in f_ans:
    #print (line)
    s = line.strip().split('\t')
    if line[0] == "#":
        l_header = s
        print (l_header)
    else:
        dic = dict(zip(l_header, s))
        #print (dic)
        sample = dic["Sample"]
        chrom = dic["#CHROM"]
        pos = dic["POS"]
        ref = dic["REF"]
        alt = dic["ALT"]
        VAF = float(dic['VAF_Validation'].rstrip('%')) * 0.01
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        validation = dic["Validation"]
        if validation == "TRUE":
            dic_ans['TRUE'].append(info)
            dic_ans['TRUE_VAF'].append(float(VAF))
        else:
            dic_ans['FALSE'].append(info)
            dic_ans['FALSE_VAF'].append(float(VAF))

for stat in dic_ans:
    print (stat, len(dic_ans[stat]))
    


print (dic_ans["TRUE"][0])
def Performance_gather(info, VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF):
    if info in dic_ans["TRUE"]:
            cnt_true +=1
            l_true.append(info)
    elif info in dic_ans["FALSE"]:
            cnt_false +=1
            l_false.append(info)
            l_false_VAF.append(VAF)
    else:
        pass
            #print (info, "Neither in TRUE nor FALSE")
    return [cnt_true, cnt_false, l_false, l_false_VAF]

from scipy.stats import hmean
def Performance_check(cnt_true, cnt_false):
    TP = cnt_true
    FP = cnt_false
    PC = len(dic_ans['TRUE'])
    NC = len(dic_ans['FALSE'])
    sens = TP/PC
    if TP == 0 and FP== 0:
        pre = None
        f_score = None
    else:
        pre = TP/(TP + FP)
        f_score = hmean([sens,pre])
    print (sens, pre, f_score)
    return (TP, FP, sens, pre, f_score)


['#CHROM', 'POS', 'REF', 'ALT', ' Mappability ', 'VAF_Validation', 'Sample', 'Group', 'Validation']
TRUE 117
FALSE 13
TRUE_VAF 117
FALSE_VAF 13
GBM-P-0396-Peri-M-Rep1:chr1:10659715:C:T


## 1. MT2-to call set + alt softclip (MF) > 0.05


In [26]:
#MT2
import glob

fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/MT2/*PASS.vcf") 
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            #print (s[8])
            VAF = s[9].split(':')[2]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]     
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_MT_false = []
for e_f in l_false:
    s = e_f.split(':')
    pos = ':'.join(s[0:3])
    print (pos)
    l_MT_false.append(pos)
    
l_MT_true = []
for e_f in l_true:
    s = e_f.split(':')
    pos = ':'.join(s[0:3])
    #print (pos)
    l_MT_true.append(pos) 


0.8547008547008547 0.9259259259259259 0.8888888888888888
(100, 8, 0.8547008547008547, 0.9259259259259259, 0.8888888888888888)
['GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'SCZ-P-1001-PFC-M-Rep1:chr13:112558343:G:A', 'NLE-P-0228-Occi-F-Rep1:chr7:100964707:C:T', 'GBM-P-0026-Tumor-F-Rep2:chr19:8371354:G:A', 'SCZ-C-1014-PFC-M-Rep2:chr2:138015377:A:G', 'ASD-P-4721-Temp-M-Rep1:chr20:46366499:G:A', 'NLE-P-0041-Frontal-F-Rep1:chr21:30425715:A:G', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
['0.279', '0.040', '0.247', '0.069', '0.057', '0.040', '0.037', '0.085']
GBM-P-0187-Tumor-M-Rep1:chr20:17456366
SCZ-P-1001-PFC-M-Rep1:chr13:112558343
NLE-P-0228-Occi-F-Rep1:chr7:100964707
GBM-P-0026-Tumor-F-Rep2:chr19:8371354
SCZ-C-1014-PFC-M-Rep2:chr2:138015377
ASD-P-4721-Temp-M-Rep1:chr20:46366499
NLE-P-0041-Frontal-F-Rep1:chr21:30425715
GBM-P-0881-Tumor-M-Rep1:chr1:110340663


In [27]:
f_MF_raw = open("/data/project/MRS/BioValidation/PlosGen/output/MF/All.snv.Predictions", 'r')

ori_fp = len(l_MT_false)
ori_tp = len(l_MT_true)
cnt_fil_tp, cnt_fil_fp = 0,0

for line in f_MF_raw:
    #print (line)
    s = line.strip().split('\t')
    if s[0] == "id":
        l_header = s
    else:
        dic = dict(zip(l_header, s))
        info_tmp = dic['id']
        VAF = dic['AF']
        info_tmp = info_tmp.lstrip('./')
        info_tmp = info_tmp.split('~')[0:3]
        #print (info_tmp.split('~'))
        pos = ':'.join(info_tmp)
        if pos in l_MT_false:
            #print (pos, dic['id'])
            alt_softclip = dic['alt_softclip']
            if float(alt_softclip) > 0.05:
                cnt_fil_fp +=1 
        if pos in l_MT_true:
            alt_softclip = dic['alt_softclip']
            if float(alt_softclip) > 0.05:
                cnt_fil_tp +=1 
                print ("FILTERED TP: ", pos)
print ("ORI TP: ", ori_tp)
print ("Filtered TP:", cnt_fil_tp)
print ("ORI FP: ", ori_fp)
print ("Filtered FP: ", cnt_fil_fp)

FILTERED TP:  GBM-P-0881-Tumor-M-Rep1:chr2:24039486
ORI TP:  100
Filtered TP: 1
ORI FP:  8
Filtered FP:  0



## 2. HC-p200 call set + MFRL alt (MT2-to) <150
## 3. HC-p200 call set + Het likelihood (MF) > 0.25

In [28]:
#HCp200
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/HCp200/*PASS.vcf")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
ori_fp = len(l_HC200_false)
ori_tp = len(l_HC200_true)
cnt_fil_tp, cnt_fil_fp = 0,0

for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_HC200_false = []
for e_f in l_false:
    s = e_f.split(':')
    pos = ':'.join(s[0:3])
    print (pos)
    l_HC200_false.append(pos)
    
l_HC200_true = []
for e_f in l_true:
    s = e_f.split(':')
    pos = ':'.join(s[0:3])
    #print (pos)
    l_HC200_true.append(pos) 

0.38461538461538464 0.9375 0.5454545454545455
(45, 3, 0.38461538461538464, 0.9375, 0.5454545454545455)
['GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T', 'NLE-P-0228-Occi-F-Rep1:chr7:100964707:C:T', 'GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A']
[0.08284023668639054, 0.245662100456621, 0.27673649393605293]
GBM-P-0881-Tumor-M-Rep1:chr1:110340663
NLE-P-0228-Occi-F-Rep1:chr7:100964707
GBM-P-0187-Tumor-M-Rep1:chr20:17456366


In [31]:
#HC200 with MT2-to MFRL alt < 150
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/MT2/*filtered.vcf")
ori_fp = len(l_MT_false)
ori_tp = len(l_MT_true)
cnt_fil_tp, cnt_fil_fp = 0,0

for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            VAF = s[9].split(':')[2]
            info = sample + ':' + chrom + ':' + pos
            if info in l_HC200_false:
                s_INFO = s[7].split(';')
                for i in s_INFO:
                    if "MFRL" in i:
                        MFRL_alt = i.lstrip("MFRL=").split(',')[1]
                        if int(MFRL_alt) < 150:
                            cnt_fil_fp +=1
            if info in l_HC200_true:
                s_INFO = s[7].split(';')
                for i in s_INFO:
                    if "MFRL" in i:
                        MFRL_alt = i.lstrip("MFRL=").split(',')[1]
                        if int(MFRL_alt) < 150:
                            cnt_fil_tp +=1
                            print ("FILTERED TP: ", pos, MFRL_alt)
                            
print ("ORI TP: ", ori_tp)
print ("Filtered TP:", cnt_fil_tp)
print ("ORI FP: ", ori_fp)
print ("Filtered FP: ", cnt_fil_fp)

FILTERED TP:  10659715 137
FILTERED TP:  238271458 146
FILTERED TP:  59994312 145
FILTERED TP:  35803613 134
FILTERED TP:  18781163 134
FILTERED TP:  10659715 134
ORI TP:  100
Filtered TP: 6
ORI FP:  8
Filtered FP:  0


In [30]:
#“Het likelihood” of MF (>0.25, 95% of adjustable true and 8% of germline false positives).

f_MF_raw = open("/data/project/MRS/BioValidation/PlosGen/output/MF/All.snv.Predictions", 'r')
ori_fp = len(l_MT_false)
ori_tp = len(l_MT_true)
cnt_fil_tp, cnt_fil_fp = 0,0
for line in f_MF_raw:
    #print (line)
    s = line.strip().split('\t')
    if s[0] == "id":
        l_header = s
    else:
        dic = dict(zip(l_header, s))
        info_tmp = dic['id']
        VAF = dic['AF']
        info_tmp = info_tmp.lstrip('./')
        info_tmp = info_tmp.split('~')[0:3]
        #print (info_tmp.split('~'))
        pos = ':'.join(info_tmp)
        if pos in l_HC200_false:
            #print (pos)
            het_likelihood = float(dic['het_likelihood'])
            if het_likelihood > 0.25:
                cnt_fil_fp +=1
        if pos in l_HC200_true:
            het_likelihood = float(dic['het_likelihood'])
            if het_likelihood > 0.25:
                print ("Filtered TP POS:", pos)
                cnt_fil_tp +=1

print ("ORI TP: ", ori_tp)
print ("Filtered TP:", cnt_fil_tp)
print ("ORI FP: ", ori_fp)
print ("Filtered FP: ", cnt_fil_fp)

ORI TP:  100
Filtered TP: 0
ORI FP:  8
Filtered FP:  0


# Paired sample analysis

### Shared variants

In [1]:
f_ans = open("../resource/Answer.shared.txt", 'r')
dic_ans = {"TRUE":[], 'FALSE':[], "TRUE_VAF" : [], "FALSE_VAF": []}
dic_shared = {'TRUE':{}, 'FALSE':{}}
#For set analysis
l_TP_set = []
l_FP_set = []
dic_ans_xVAF = {}

for line in f_ans:
    #print (line)
    s = line.strip().split('\t')
    if line[0] == "X":
        l_header = s
        print (l_header)
    else:
        dic = dict(zip(l_header, s))
        #print (dic)
        sample = dic["Sample"]
        chrom = dic["X.CHROM"]
        pos = dic["POS"]
        ref = dic["REF"]
        alt = dic["ALT"]
        VAF = float(dic['VAF_Validation'].rstrip('%')) *0.01

        locus = chrom + ':' + pos
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        dic_ans_xVAF[info] = VAF * 100
            
        validation = dic["Validation"]
        if validation == "TRUE":
            dic_ans['TRUE'].append(info)
            dic_ans['TRUE_VAF'].append(float(VAF))
            l_TP_set.append(info)
            if locus not in dic_shared["TRUE"]:
                dic_shared['TRUE'][locus] = [VAF*100]
            else:
                dic_shared['TRUE'][locus].append(VAF*100)
        else:
            dic_ans['FALSE'].append(info)
            dic_ans['FALSE_VAF'].append(float(VAF))
            l_FP_set.append(info)
            if locus not in dic_shared["FALSE"]:
                dic_shared['FALSE'][locus] = [VAF*100]
            else:
                dic_shared['FALSE'][locus].append(VAF*100)

for stat in dic_ans:
    print (stat, len(dic_ans[stat]))
    
#### REmove shared that one of them was valiated as false  
dic_shared2 = {'TRUE':{}, 'FALSE':{}}
for valid in dic_shared:
    for locus in dic_shared[valid]:
        if len(dic_shared[valid][locus]) != 1:
            dic_shared2[valid][locus] = dic_shared[valid][locus]
print ("DIC_SHARED2", dic_shared2)
dic_shared = dic_shared2 
    

print (dic_ans["TRUE"][0])
def Performance_gather(info, VAF, cnt_true, cnt_false, dic_true,dic_false):
    variant = info.split(':')[1] + ':' + info.split(':')[2]
    if variant in dic_shared["TRUE"]:
            cnt_true +=1
            if variant not in dic_true:
                dic_true[variant] = [VAF]
            else:
                dic_true[variant].append(VAF)
    elif variant in dic_shared["FALSE"]:
            cnt_false +=1
            if variant not in dic_false:
                dic_false[variant] = [VAF]
            else:
                dic_false[variant].append(VAF)
    else:
        pass
            #print (info, "Neither in TRUE nor FALSE")
    return [cnt_true, cnt_false, dic_true, dic_false]

from scipy.stats import hmean
def Performance_check(cnt_true, cnt_false):
    TP = cnt_true
    FP = cnt_false
    PC = len(dic_ans['TRUE'])
    NC = len(dic_ans['FALSE'])
    sens = TP/PC
    if TP == 0 and FP== 0:
        pre = None
        f_score = None
    else:
        pre = TP/(TP + FP)
        f_score = hmean([sens,pre])
    print (sens, pre, f_score)
    return (TP, FP, sens, pre, f_score)

for e_f in dic_ans['FALSE']:
    print (e_f)
print (dic_ans['FALSE'])

['X.CHROM', 'POS', 'REF', 'ALT', 'Mappability', 'VAF_Validation', 'Sample', 'Group', 'Validation', 'VAF', 'SET', 'info', 'n']
TRUE 51
FALSE 5
TRUE_VAF 51
FALSE_VAF 5
DIC_SHARED2 {'TRUE': {'chr1:10659715': [22.07, 21.83], 'chr1:121183580': [8.5, 8.5], 'chr1:152308217': [2.6, 2.9, 2.9, 2.5], 'chr2:111858311': [3.6000000000000005, 3.6000000000000005], 'chr2:219420155': [4.2, 4.2], 'chr5:733811': [6.5, 6.4, 6.4, 6.800000000000001], 'chr5:837373': [5.4, 5.4], 'chr6:136192149': [7.000000000000001, 7.000000000000001], 'chr7:23585592': [24.5, 28.4], 'chr7:55157725': [2.5, 2.5], 'chr9:127676696': [1.8000000000000003, 1.7000000000000002], 'chr12:389107': [8.5, 8.0, 7.6, 8.1, 7.6, 7.8], 'chr12:122701415': [8.4, 8.4], 'chr12:122701470': [8.1, 9.5, 8.1], 'chr15:74722831': [2.3, 2.3], 'chr16:27749710': [7.400000000000001, 7.400000000000001], 'chr16:69463844': [3.3000000000000003, 3.3000000000000003], 'chr17:18781163': [10.6, 10.4], 'chr20:36209646': [4.5, 4.5], 'chr21:44550435': [7.400000000000001, 

In [4]:
## Shared VAF combination
l_result = []
out = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_result.paired.shared.txt', 'w')
out.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" %("tool","range", "PC", "TP", "FP", "Sensitivity", "Precision", "F1-score"))
from scipy.stats import hmean
cnt_sha = 0
for pos in dic_shared['TRUE']:
    cnt_sha += len(dic_shared['TRUE'][pos])
print ("TRUE Total shared var count: ", cnt_sha)

cnt_sha = 0
for pos in dic_shared['FALSE']:
    cnt_sha += len(dic_shared['FALSE'][pos])
print ("FALSE Total shared var count: ", cnt_sha)



TRUE Total shared var count:  51
FALSE Total shared var count:  5


In [7]:
import itertools


def Parse_in_comb(dic):
    dic_template = {}
    Misclassified = 0
    l_range = [0,5,10,25,90]
    for i in range(len(l_range)-1):
        #print (i)
        for j in range(len(l_range)-1):
            if i>=j:
                dic_template[str(l_range[i+1])+':'+str(l_range[j+1])] = 0
    for var in dic:
        l_vaf = dic[var]
        if len(l_vaf) > 1:
            l_vaf_comb = (list(itertools.combinations(l_vaf,2)))
            #print ("L_VAF", l_vaf, l_vaf_comb)

            for i in l_vaf_comb:
                a = float(i[0]) 
                b = float(i[1]) 
                if b > a:
                    a,b = b,a
                a_stat = None
                b_stat = None
                for j in range(len(l_range)-1):
                    if l_range[j] <= a < l_range[j+1]:
                        a_stat = l_range[j+1]
                    if l_range[j] <= b < l_range[j+1]:
                        b_stat = l_range[j+1]
                #print (a, b, a_stat, b_stat)
                #if a_stat == None:
                 #   a_stat = 90
                #if b_stat ==None:
                #    b_stat = 90
                #print (a, a_stat)
                #print (b, b_stat)
                vaf_comb = str(a_stat) + ':' + str(b_stat)
                #print (vaf_comb)
                dic_template[vaf_comb] +=1
        elif len(l_vaf) == 1:
            Misclassified +=1
            print (var, l_vaf)
    print ("MISCLASSIFIED CALLS: ", Misclassified)
        
    return dic_template

def Parse_in_comb_Mis(dic):
    dic_Mis = {}
    Misclassified = 0
    l_range = [0,5,10,25,90]
    for i in range(len(l_range)-1):
        #print (i)
        for j in range(len(l_range)-1):
            if i>=j:
                dic_template[str(l_range[i+1])+':'+str(l_range[j+1])] = 0
    for var in dic:
        l_vaf = dic[var]

        


dic_PC = dic_shared["TRUE"]
print ("DIC_PC", dic_PC)
print ("DIC_FP", dic_shared["FALSE"])
dic_vafcomb = {}
dic_vafcomb = Parse_in_comb(dic_PC)

print (dic_vafcomb)

def dic_with_VAFcomb(dic):
    l_range = [0,5,10,25,90]
    for i in range(len(l_range)-1):
        #print (i)
        for j in range(len(l_range)-1):
            if i>=j:
                dic[str(l_range[i+1])+':'+str(l_range[j+1])] = 0
    return dic

print (dic_shared['FALSE'])
print (Parse_in_comb(dic_shared['FALSE']))

DIC_PC {'chr1:10659715': [22.07, 21.83], 'chr1:121183580': [8.5, 8.5], 'chr1:152308217': [2.6, 2.9, 2.9, 2.5], 'chr2:111858311': [3.6000000000000005, 3.6000000000000005], 'chr2:219420155': [4.2, 4.2], 'chr5:733811': [6.5, 6.4, 6.4, 6.800000000000001], 'chr5:837373': [5.4, 5.4], 'chr6:136192149': [7.000000000000001, 7.000000000000001], 'chr7:23585592': [24.5, 28.4], 'chr7:55157725': [2.5, 2.5], 'chr9:127676696': [1.8000000000000003, 1.7000000000000002], 'chr12:389107': [8.5, 8.0, 7.6, 8.1, 7.6, 7.8], 'chr12:122701415': [8.4, 8.4], 'chr12:122701470': [8.1, 9.5, 8.1], 'chr15:74722831': [2.3, 2.3], 'chr16:27749710': [7.400000000000001, 7.400000000000001], 'chr16:69463844': [3.3000000000000003, 3.3000000000000003], 'chr17:18781163': [10.6, 10.4], 'chr20:36209646': [4.5, 4.5], 'chr21:44550435': [7.400000000000001, 9.7], 'chr22:38670902': [4.0, 4.0]}
DIC_FP {'chr5:173609431': [1.8000000000000003, 1.8000000000000003, 1.8000000000000003], 'chr12:103985722': [5.4, 5.4]}
MISCLASSIFIED CALLS:  0
{

In [8]:
#MF
f = open('/data/project/MRS/BioValidation/PlosGen/output/MF/All.snv.Predictions.Mosaic', 'r')
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0
dic_TP = dic_with_VAFcomb({})
dic_FP = dic_with_VAFcomb({})

for line in f:
    #print (line)
    s = line.strip().split('\t')
    if s[0] == "id":
        l_header = s
    else:
        dic = dict(zip(l_header, s))
        #print (dic)
        info_tmp = dic['id']

        info_tmp = info_tmp.split('/')[-1]
        #print (info_tmp.split('~'))
        info = ':'.join(info_tmp.split('~'))
        if info in dic_ans_xVAF:
            VAF =  dic_ans_xVAF[info]
        #print (info) #AD-P-2013029-HIF-F-Rep1:chr8:123182166:A:G
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,dic_true, dic_false)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            dic_true = cnts[2]
            dic_false = cnts[3]
        
print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['MF', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')
    #print (l_result_tmp)



0.35294117647058826 1.0 0.5217391304347826
(18, 0, 0.35294117647058826, 1.0, 0.5217391304347826)
DIC_TRUE: {'chr15:74722831': [2.3, 2.3], 'chr16:69463844': [3.3000000000000003, 3.3000000000000003], 'chr20:36209646': [4.5, 4.5], 'chr9:127676696': [1.8000000000000003, 1.7000000000000002], 'chr1:10659715': [22.07, 21.83], 'chr2:219420155': [4.2, 4.2], 'chr6:136192149': [7.000000000000001, 7.000000000000001], 'chr16:27749710': [7.400000000000001, 7.400000000000001], 'chr22:38670902': [4.0, 4.0]}
DIC_FALSE:  {}
MISCLASSIFIED CALLS:  0
TP {'5:5': 6, '10:5': 0, '10:10': 2, '25:5': 0, '25:10': 0, '25:25': 1, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
MISCLASSIFIED CALLS:  0
10:5
MISCLASSIFIED CALLS:  0


In [9]:
#MT2
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/MT2/*PASS.vcf")
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0

for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            #print (s[8])
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            if info in dic_ans_xVAF:
                VAF =  dic_ans_xVAF[info]
                cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false,dic_true, dic_false)
                cnt_true = cnts[0]
                cnt_false = cnts[1]
                dic_true = cnts[2]
                dic_false = cnts[3]

print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['MT2', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')
    #print (l_result_tmp)


0.6862745098039216 1.0 0.813953488372093
(35, 0, 0.6862745098039216, 1.0, 0.813953488372093)
DIC_TRUE: {'chr1:152308217': [2.9, 2.5, 2.9], 'chr12:389107': [8.1, 8.0, 7.8, 7.6, 8.5], 'chr12:122701415': [8.4, 8.4], 'chr16:27749710': [7.400000000000001, 7.400000000000001], 'chr9:127676696': [1.7000000000000002, 1.8000000000000003], 'chr1:10659715': [22.07, 21.83], 'chr7:23585592': [24.5, 28.4], 'chr7:55157725': [2.5, 2.5], 'chr16:69463844': [3.3000000000000003, 3.3000000000000003], 'chr20:36209646': [4.5], 'chr22:38670902': [4.0], 'chr17:18781163': [10.6, 10.4], 'chr2:219420155': [4.2, 4.2], 'chr6:136192149': [7.000000000000001, 7.000000000000001], 'chr15:74722831': [2.3, 2.3], 'chr12:122701470': [9.5, 8.1, 8.1]}
DIC_FALSE:  {}
chr20:36209646 [4.5]
chr22:38670902 [4.0]
MISCLASSIFIED CALLS:  2
TP {'5:5': 8, '10:5': 0, '10:10': 16, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10'

In [10]:
#DM
f = open('/data/project/MRS/BioValidation/PlosGen/output/DM_output/All.DM.output.mosaic.txt', 'r')
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0

for line in f:
    #print (line)
    s = line.strip().split('\t')
    sample = s[0]
    chrom = s[2]
    pos = s[3]
    ref = s[4]
    alt = s[5]
    #print (VAF) *100
    info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
    if info in dic_ans_xVAF:
        VAF =  dic_ans_xVAF[info]
        cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
        cnt_true = cnts[0]
        cnt_false = cnts[1]
        dic_true = cnts[2]
        dic_false = cnts[3]

print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['DM', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')

0.0 None None
(0, 0, 0.0, None, None)
DIC_TRUE: {}
DIC_FALSE:  {}
MISCLASSIFIED CALLS:  0
TP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
MISCLASSIFIED CALLS:  0
10:5
MISCLASSIFIED CALLS:  0
10:10
MISCLASSIFIED CALLS:  0
25:5
MISCLASSIFIED CALLS:  0
25:10
MISCLASSIFIED CALLS:  0
25:25
MISCLASSIFIED CALLS:  0
90:5
MISCLASSIFIED CALLS:  0
90:10
MISCLASSIFIED CALLS:  0
90:25
MISCLASSIFIED CALLS:  0
90:90
MISCLASSIFIED CALLS:  0


In [11]:
#HCp20
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/HCp20/*PASS.vcf")
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0

for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #print (info)
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            if info in dic_ans_xVAF:
                VAF =  dic_ans_xVAF[info]
                cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
                cnt_true = cnts[0]
                cnt_false = cnts[1]
                dic_true = cnts[2]
                dic_false = cnts[3]
    
print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['HCp20', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')

0.1568627450980392 1.0 0.2711864406779661
(8, 0, 0.1568627450980392, 1.0, 0.2711864406779661)
DIC_TRUE: {'chr7:23585592': [24.5, 28.4], 'chr17:18781163': [10.6], 'chr1:10659715': [22.07, 21.83], 'chr9:127676696': [1.7000000000000002, 1.8000000000000003], 'chr12:122701470': [9.5]}
DIC_FALSE:  {}
chr17:18781163 [10.6]
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  2
TP {'5:5': 1, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 1, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
chr17:18781163 [10.6]
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  2
10:5
chr17:18781163 [10.6]
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  2
10:10
chr17:18781163 [10.6]
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  2
25:5
chr17

In [12]:
#HCp200

import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/HCp200/*PASS.vcf")
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0

for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #print (info)
            if info in dic_ans_xVAF:
                VAF =  dic_ans_xVAF[info]
                cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
                cnt_true = cnts[0]
                cnt_false = cnts[1]
                dic_true = cnts[2]
                dic_false = cnts[3]
    
print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['HCp200', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')

0.17647058823529413 1.0 0.30000000000000004
(9, 0, 0.17647058823529413, 1.0, 0.30000000000000004)
DIC_TRUE: {'chr7:23585592': [28.4, 24.5], 'chr17:18781163': [10.6, 10.4], 'chr1:10659715': [21.83, 22.07], 'chr12:122701470': [9.5], 'chr9:127676696': [1.8000000000000003, 1.7000000000000002]}
DIC_FALSE:  {}
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  1
TP {'5:5': 1, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  1
10:5
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  1
10:10
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  1
25:5
chr12:122701470 [9.5]
MISCLASSIFIED CALLS:  1
25:10
chr12:122701470 [9.5]
MISCLASSI

In [13]:
#MH
import glob
fl = glob.glob('/data/project/MRS/BioValidation/PlosGen/output/MH/*/run/final.passed.tsv')
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0
for file in fl:
    f = open(file, 'r')
    sample = str(file).split('/')[-3]
    for line in f:
        s = line.strip().split('\t')
        chrom = s[0]
        pos = s[1]
        ref = s[6]
        alt = s[8]
       # VAF = float(s[9])*100/int(s[7])
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        if info in dic_ans_xVAF:
            VAF =  dic_ans_xVAF[info]
            cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            dic_true = cnts[2]
            dic_false = cnts[3]
    
print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['MH', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')
    


0.0392156862745098 1.0 0.07547169811320754
(2, 0, 0.0392156862745098, 1.0, 0.07547169811320754)
DIC_TRUE: {'chr16:27749710': [7.400000000000001], 'chr20:36209646': [4.5]}
DIC_FALSE:  {}
chr16:27749710 [7.400000000000001]
chr20:36209646 [4.5]
MISCLASSIFIED CALLS:  2
TP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
chr16:27749710 [7.400000000000001]
chr20:36209646 [4.5]
MISCLASSIFIED CALLS:  2
10:5
chr16:27749710 [7.400000000000001]
chr20:36209646 [4.5]
MISCLASSIFIED CALLS:  2
10:10
chr16:27749710 [7.400000000000001]
chr20:36209646 [4.5]
MISCLASSIFIED CALLS:  2
25:5
chr16:27749710 [7.400000000000001]
chr20:36209646 [4.5]
MISCLASSIFI

## Joint genotyping methods

In [14]:
#MT2-Modified
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/Paired/MT2PM/*normal.vcf")
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0

l_uniq_control = []
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
            control_sample = line.strip().split('\t')[-2]
        else:
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            #print (s[8])
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            if info not in l_uniq_control:
                if info in dic_ans_xVAF:
                    VAF =  dic_ans_xVAF[info]
                    cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false,dic_true, dic_false)
                    cnt_true = cnts[0]
                    cnt_false = cnts[1]
                    dic_true = cnts[2]
                    dic_false = cnts[3]
                    l_uniq_control.append(info)
            ##COntrol
            info = control_sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            if info not in l_uniq_control:
                if info in dic_ans_xVAF:
                    VAF =  dic_ans_xVAF[info]
                    cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false,dic_true, dic_false)
                    cnt_true = cnts[0]
                    cnt_false = cnts[1]
                    dic_true = cnts[2]
                    dic_false = cnts[3]
                    l_uniq_control.append(info)

print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)
print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    #print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['MT2m', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')
    print (l_result_tmp)


0.47058823529411764 1.0 0.64
(24, 0, 0.47058823529411764, 1.0, 0.64)
DIC_TRUE: {'chr12:389107': [7.8, 8.5, 8.1, 7.6, 7.6, 8.0], 'chr22:38670902': [4.0, 4.0], 'chr7:55157725': [2.5, 2.5], 'chr2:219420155': [4.2, 4.2], 'chr6:136192149': [7.000000000000001, 7.000000000000001], 'chr20:36209646': [4.5, 4.5], 'chr16:69463844': [3.3000000000000003, 3.3000000000000003], 'chr15:74722831': [2.3, 2.3], 'chr21:44550435': [9.7, 7.400000000000001], 'chr5:837373': [5.4, 5.4]}
DIC_FALSE:  {}
MISCLASSIFIED CALLS:  0
TP {'5:5': 6, '10:5': 0, '10:10': 18, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
MISCLASSIFIED CALLS:  0
['MT2m', '5:5', 14, 6, 0, 0.42857142857142855, 1.0, 0.6]
MISCLA

In [15]:
#MH-Modified
import glob
fl = glob.glob('/data/project/MRS/BioValidation/PlosGen/output/Paired/MHP/*/final.passed.tsv')
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0
for file in fl:
    f = open(file, 'r')
    sample = str(file).split('/')[-2]
    sample = sample.split('_')[1]
    print (sample)
    for line in f:
        s = line.strip().split('\t')
        chrom = s[0]
        pos = s[1]
        ref = s[6]
        alt = s[8]
        joint_mosaic = 10**float(s[43])
        print ("Joint Mosaic: ", joint_mosaic)
        biggest = 0
        for i in range(28,43):
            each_joint = 10**float(s[i])
            if each_joint > biggest:
                biggest = each_joint
        print ("Biggest", biggest)
        if biggest < joint_mosaic and joint_mosaic > 0.05:
            print ("YES MOSAIC", joint_mosaic, biggest)
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            if info in dic_ans_xVAF:
                VAF =  dic_ans_xVAF[info]
                cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
                cnt_true = cnts[0]
                cnt_false = cnts[1]
                dic_true = cnts[2]
                dic_false = cnts[3]

print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['MHPm', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')

GBM-P-0026-Tumor-F-Rep1
Joint Mosaic:  1.0
Biggest 1.8964438997045733e-31
YES MOSAIC 1.0 1.8964438997045733e-31
Joint Mosaic:  0.9445827277856884
Biggest 0.05501739205107149
YES MOSAIC 0.9445827277856884 0.05501739205107149
Joint Mosaic:  1.0
Biggest 1.4039446945093801e-07
YES MOSAIC 1.0 1.4039446945093801e-07
ASD-C-5309-Temp-F-Rep1
GBM-P-0187-SVZ-M-Rep1
SCZ-C-0007-PFC-M-Rep1
Joint Mosaic:  1.0
Biggest 2.0171119393771587e-32
YES MOSAIC 1.0 2.0171119393771587e-32
ASD-P-4999-Heart-M-Rep1
NLE-P-0150-Brain-M-Rep1
ASD-C-5408-Temp-M-Rep2
ASD-P-4999-Occi-M-Rep1
GBM-P-0396-Recur-M-Rep1
Joint Mosaic:  1.0
Biggest 6.615154625040279e-29
YES MOSAIC 1.0 6.615154625040279e-29
Joint Mosaic:  0.003403768383918788
Biggest 0.9965979741107903
ASD-P-5771-Occi-F-Rep1
ASD-C-5408-Occi-M-Rep1
SCZ-P-1002-PFC-M-Rep1
ASD-P-5144-Cereb-M-Rep1
ASD-P-5574-Occi-M-Rep1
ASD-C-5408-Heart-M-Rep1
SCZ-C-0002-PFC-M-Rep1
ASD-C-5408-PFC-M-Rep1
ASD-P-1182-Temp-F-Rep1
FCD-P-0023-Lhem-M-Rep1
FCD-P-0021-RTemOcci-M-Rep1
Joint Mosa

In [16]:
#M2S2MH
import glob
fl = glob.glob('/data/project/MRS/BioValidation/PlosGen/output/Paired/MH_rescue/MH_rescue.txt*')
dic_true,dic_false, cnt_true, cnt_false = {}, {},0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        s = line.strip().split('\t')
        sample = s[0]
        chrom = s[1]
        pos = s[2]
        ref = s[3]
        alt = s[4]
        
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        if info in dic_ans_xVAF:
            VAF =  dic_ans_xVAF[info]
            cnts = Performance_gather(info, float(VAF), cnt_true, cnt_false, dic_true, dic_false)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            dic_true = cnts[2]
            dic_false = cnts[3]

print (Performance_check(cnt_true, cnt_false))
print ("DIC_TRUE:", dic_true)
print ("DIC_FALSE: ", dic_false)

print ("TP", Parse_in_comb(dic_true))
print ("FP", Parse_in_comb(dic_false))

dic_vafcomb =  Parse_in_comb(dic_shared['TRUE'])
print ("PC", dic_vafcomb)


for each in dic_vafcomb:
    print (each)
    PC = int(dic_vafcomb[each])
    TP=int(Parse_in_comb(dic_true)[each])
    if len(dic_false) == 0:
        FP = 0
    else:
        FP =int(Parse_in_comb(dic_false)[each])
    if PC == 0:
        sen, pre, fscore = None, None, None
    else:
        sen = TP/float(PC)
        if TP != 0:
            
            pre = float(TP)/(TP + FP)
            fscore = hmean([sen,pre])
        else:
            pre = None
            fscore = None
    l_result_tmp = ['M2S2MH', each, PC, TP, FP, sen, pre, fscore]
   
    out.write('\t'.join(map(str, l_result_tmp))+ '\n')

0.0196078431372549 1.0 0.038461538461538464
(1, 0, 0.0196078431372549, 1.0, 0.038461538461538464)
DIC_TRUE: {'chr16:27749710': [7.400000000000001]}
DIC_FALSE:  {}
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
TP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
FP {'5:5': 0, '10:5': 0, '10:10': 0, '25:5': 0, '25:10': 0, '25:25': 0, '90:5': 0, '90:10': 0, '90:25': 0, '90:90': 0}
MISCLASSIFIED CALLS:  0
PC {'5:5': 14, '10:5': 0, '10:10': 30, '25:5': 0, '25:10': 0, '25:25': 2, '90:5': 0, '90:10': 0, '90:25': 1, '90:90': 0}
5:5
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
10:5
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
10:10
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
25:5
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
25:10
chr16:27749710 [7.400000000000001]
MISCLASSIFIED CALLS:  1
25:25
chr16:27749710 [7.400000000000001]
MISCLASSI

## Best block of shared variants


In [547]:
f = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_result.paired.shared.txt', 'r')
out = open('/data/project/MRS/BioValidation/PlosGen/analysis/Parse_result.paired.shared.BEST.txt', 'w')
dic_best = {}
for line in f:
    s = line.strip().split('\t')
    Tool = s[0]
    if Tool == 'tool':
        l_header = s
        print (l_header)
        out.write(line)
    else:
        dic= dict(zip(l_header, s))
        Range = dic['range']
        if dic['PC'] != '0':
            if dic['F1-score'] != "None":
                if Range not in dic_best:
                    dic_best[Range] = line
                else:
                    current = dic_best[Range]
                    print ("dic", dic['F1-score'])
                    current_fscore = current.strip().split('\t')[-1]
                    print ('current', current_fscore)
                    if current_fscore != None and dic['F1-score'] != None:
                        if float(current_fscore) < float(dic['F1-score']):
                            dic_best[Range] = line
                        elif float(current_fscore) == float(dic['F1-score']):
                            s = line.split('\t')
                            tool = s[0]
                            current_tool = current.strip().split('\t')[0]
                            tools = current_tool + ';' + tool
                            s[0] = tools
                            dic_best[Range] = '\t'.join(s)
                            print ("Same")
        else:
            dic_best[Range] = line
            
    #print (line)
for Range in dic_best:
    e_l = dic_best[Range]
    PC = e_l.split('\t')[3]
    s = e_l.split('\t')
    if PC == '0':
        s[0] = 'NA'
        out.write('\t'.join(s))
    else:
        out.write('\t'.join(s))
    print (Range, dic_best[Range])

['tool', 'range', 'PC', 'TP', 'FP', 'Sensitivity', 'Precision', 'F1-score']
dic 0.7272727272727273
current 0.6
dic 0.6956521739130435
current 0.125
dic 1.0
current 0.6666666666666666
dic 0.13333333333333333
current 0.7272727272727273
dic 0.6666666666666666
current 1.0
dic 1.0
current 1.0
Same
dic 0.13333333333333333
current 0.7272727272727273
dic 1.0
current 1.0
Same
dic 1.0
current 1.0
Same
dic 0.6
current 0.7272727272727273
dic 0.7499999999999999
current 0.6956521739130435
5:5 MT2	5:5	14	8	0	0.5714285714285714	1.0	0.7272727272727273

10:5 M2S2MH	10:5	0	0	0	None	None	None

10:10 MT2m	10:10	30	18	0	0.6	1.0	0.7499999999999999

25:5 M2S2MH	25:5	0	0	0	None	None	None

25:10 M2S2MH	25:10	0	0	0	None	None	None

25:25 MT2;HCp200	25:25	2	2	0	1.0	1.0	1.0

90:5 M2S2MH	90:5	0	0	0	None	None	None

90:10 M2S2MH	90:10	0	0	0	None	None	None

90:90 M2S2MH	90:90	0	0	0	None	None	None

90:25 MT2;HCp20;HCp200	90:25	1	1	0	1.0	1.0	1.0



## RESCUE MF

In [18]:
def get_HQ(chrom,pos,sample):
    f = open("/data/project/MRS/BioValidation/PlosGen/analysis/Rescue/input/PU.vcf", 'r')
    for line in f:
        if line[:2] == '#C':
            #print (line)
            l_header = line.strip().split('\t')
        elif line[0] != '#':
            s = line.strip().split('\t')
            dic = dict(zip(l_header, s))
            if chrom == dic['#CHROM'] and pos == dic['POS']:
                FORMAT = dic[sample]
                AD = FORMAT.split(':')
                alt = int(AD[1].split(',')[1])
    return (alt)
            
print (get_HQ('chr16', '27749710', 'SCZ-C-1021-Liver-M-Rep1'))
print (get_HQ('chr16', '27749710', 'SCZ-C-1021-PFC-M-Rep2'))

import glob
dic_pair = {}
f_ss = open("/data/project/MRS/BioValidation/PlosGen/resource/Answer.paired.shared.txt", 'r')
for line in f_ss:
    s = line.strip().split('\t')
    if s[0] != 'X.CHROM':
        sample = s[6]
        s_ID = sample.split('-')
        ID = '-'.join(s_ID[:3])
        vaf = float(s[5].rstrip("%"))
        
        #print (ID) #GBM-P-0396
        #print (sample)
        locus = s[0] + ':' + s[1]
        if locus not in dic_pair:
            dic_pair[locus] = {sample:vaf}
        elif locus in dic_pair:
            dic_pair[locus][sample] = vaf
print (dic_pair)

f_counter = open("/data/project/MRS/BioValidation/PlosGen/output/MF/All.snv.Predictions.Mosaic", 'r')
rl_counter = f_counter.readlines()
dic_locus_sample = {}
for line_counter in rl_counter:
    if 'id' not in line_counter:
        info_tmp = line_counter.split('\t')[0]
        info_tmp = info_tmp.split('/')[-1]
        info = info_tmp.split('~')
        tmp_locus = info[1] + ':' + info[2]
        tmp_sample = info[0]
        if tmp_locus not in dic_locus_sample:
            dic_locus_sample[tmp_locus] = [tmp_sample]
        else:
            dic_locus_sample[tmp_locus].append(tmp_sample)
print (dic_locus_sample)
            
import itertools
for locus in dic_pair:
    l_sample = dic_pair[locus]
    pairs = list(itertools.combinations(l_sample,2))
    for each_comb in pairs:
        sample_1 = each_comb[0]
        sample_2 = each_comb[1]
        sample1_stat = 0
        sample2_stat = 0
        if locus in dic_locus_sample:
            if sample_1  in dic_locus_sample[locus]:
                sample1_stat = 1
            if sample_2 in dic_locus_sample[locus]:
                sample2_stat = 1
            
        print (locus, sample_1, sample_2, sample1_stat, sample2_stat)
        

20
11
{'chr1:10659715': {'GBM-P-0396-Peri-M-Rep1': 22.07, 'GBM-P-0396-Recur-M-Rep1': 21.83}, 'chr1:121183580': {'ASD-C-5309-Occi-F-Rep1': 8.5, 'ASD-C-5309-Temp-F-Rep1': 8.5}, 'chr1:152308217': {'ASD-C-5408-Heart-M-Rep1': 2.6, 'ASD-C-5408-Occi-M-Rep1': 2.9, 'ASD-C-5408-PFC-M-Rep1': 2.9, 'ASD-C-5408-Temp-M-Rep1': 2.5}, 'chr2:111858311': {'GBM-P-0026-SVZ-F-Rep1': 3.6, 'GBM-P-0026-Tumor-F-Rep2': 3.6}, 'chr2:219420155': {'SCZ-C-0002-Liver-M-Rep1': 4.2, 'SCZ-C-0002-PFC-M-Rep1': 4.2}, 'chr5:733811': {'ASD-P-4999-Heart-M-Rep1': 6.5, 'ASD-P-4999-Occi-M-Rep1': 6.4, 'ASD-P-4999-PFC-M-Rep1': 6.4, 'ASD-P-4999-Temp-M-Rep1': 6.8}, 'chr5:837373': {'NLE-P-0150-Blood-M-Rep1': 5.4, 'NLE-P-0150-Brain-M-Rep1': 5.4}, 'chr5:173609431': {'ASD-C-5408-Occi-M-Rep1': 1.8, 'ASD-C-5408-PFC-M-Rep1': 1.8, 'ASD-C-5408-Temp-M-Rep2': 1.8}, 'chr6:136192149': {'SCZ-C-0002-Liver-M-Rep1': 7.0, 'SCZ-C-0002-PFC-M-Rep1': 7.0}, 'chr7:23585592': {'GBM-P-0026-Tumor-F-Rep1': 24.5, 'GBM-P-0026-Tumor-F-Rep2': 28.4}, 'chr7:55157725':

# Paired sample analysis - Sample-specific variants parsing


In [17]:
f_ans = open("../resource/Answer.paired.ss.txt", 'r')
dic_ans = {"TRUE":[], 'FALSE':[], "TRUE_VAF" : [], "FALSE_VAF": []}

#For set analysis
l_TP_set = []
l_FP_set = []


for line in f_ans:
    #print (line)
    s = line.strip().split('\t')
    if line[0] == "X":
        l_header = s
        print (l_header)
    else:
        dic = dict(zip(l_header, s))
        #print (dic)
        sample = dic["Sample"]
        chrom = dic["X.CHROM"]
        pos = dic["POS"]
        ref = dic["REF"]
        alt = dic["ALT"]
        VAF = float(dic['VAF_Validation'].rstrip('%')) * 0.01
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        shared_stat = None

        validation = dic["Validation"]
        if validation == "TRUE":
            dic_ans['TRUE'].append(info)
            dic_ans['TRUE_VAF'].append(float(VAF))
            l_TP_set.append(info)
        else:
            dic_ans['FALSE'].append(info)
            dic_ans['FALSE_VAF'].append(float(VAF))
            l_FP_set.append(info)

for stat in dic_ans:
    print (stat, len(dic_ans[stat]))
    

print (dic_ans["TRUE"][0])
def Performance_gather(info, VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF):
    if info in dic_ans["TRUE"]:
            cnt_true +=1
            l_true.append(info)
    elif info in dic_ans["FALSE"]:
            cnt_false +=1
            l_false.append(info)
            l_false_VAF.append(VAF)
    else:
        pass
            #print (info, "Neither in TRUE nor FALSE")
    return [cnt_true, cnt_false, l_false, l_false_VAF]

from scipy.stats import hmean
def Performance_check(cnt_true, cnt_false):
    TP = cnt_true
    FP = cnt_false
    PC = len(dic_ans['TRUE'])
    NC = len(dic_ans['FALSE'])
    sens = TP/PC
    if TP == 0 and FP== 0:
        pre = None
        f_score = None
    else:
        pre = TP/(TP + FP)
        f_score = hmean([sens,pre])
    print (sens, pre, f_score)
    return (TP, FP, sens, pre, f_score)


['X.CHROM', 'POS', 'REF', 'ALT', 'Mappability', 'VAF_Validation', 'Sample', 'Group', 'Validation', 'VAF', 'SET', 'info', 'n']
TRUE 25
FALSE 3
TRUE_VAF 25
FALSE_VAF 3
GBM-P-0396-Recur-M-Rep1:chr1:116031514:G:A


In [515]:
#MF
f = open('/data/project/MRS/BioValidation/PlosGen/output/SS/MF/All.ss.tsv', 'r')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0

for line in f:
    #print (line)
    s = line.strip().split('\t')
    if s[0] == "id":
        l_header = s
    else:
        dic = dict(zip(l_header, s))
        info_tmp = dic['id']
        VAF = dic['AF']
        info_tmp = info_tmp.split('/')[-1]
        #print (info_tmp.split('~'))
        info = ':'.join(info_tmp.split('~'))
        #print (info) #AD-P-2013029-HIF-F-Rep1:chr8:123182166:A:G
        cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
        cnt_true = cnts[0]
        cnt_false = cnts[1]
        l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_true)
print (l_false)
print (l_false_VAF)



l_result_tmp = ['MF']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


0.56 0.875 0.6829268292682927
(14, 2, 0.56, 0.875, 0.6829268292682927)
['GBM-P-0881-Tumor-M-Rep1:chr2:24039486:C:A', 'GBM-P-0881-Tumor-M-Rep1:chr2:218164082:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr3:197090990:C:T', 'GBM-P-0881-Tumor-M-Rep1:chr11:66332605:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr11:129864369:T:C', 'GBM-P-0881-Tumor-M-Rep1:chr15:94479623:A:T', 'GBM-P-0881-Tumor-M-Rep1:chr16:54112977:G:T', 'GBM-P-0881-Tumor-M-Rep1:chr17:7675094:A:C', 'GBM-P-0881-Tumor-M-Rep1:chr17:32489134:C:A', 'GBM-P-0187-Tumor-M-Rep1:chr5:113434693:G:A', 'GBM-P-0026-Tumor-F-Rep2:chr11:128480240:G:A', 'GBM-P-0026-Tumor-F-Rep2:chr21:33262947:C:A', 'GBM-P-0052-Tumor-M-Rep1:chr12:89425318:A:T', 'GBM-P-0052-SVZ-M-Rep2:chr17:9969721:C:G']
['GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T', 'GBM-P-0026-Tumor-F-Rep2:chr19:8371354:G:A']
['0.0835913312693498', '0.0607142857142857']
0.56 0.875 0.6829268292682927


55

In [516]:
#MT2
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/MT2/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            print (s[8])
            VAF = s[9].split(':')[2]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['MT2']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
0.52 0.8125 0.6341463414634146
(13, 3, 0.52, 0.8125, 0.6341463414634146)
['GBM-P-0026-Tumor-F-Rep2:chr19:8371354:G:A', 'GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
['0.069', '0.279', '0.085']
0.52 0.8125 0.6341463414634146


57

In [517]:
#DM
f = open('/data/project/MRS/BioValidation/PlosGen/output/SS/DM/All.ss.tsv', 'r')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0

for line in f:
    #print (line)
    s = line.strip().split('\t')
    sample = s[0]
    chrom = s[2]
    pos = s[3]
    ref = s[4]
    alt = s[5]
    VAF = s[7]
    info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt

    cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
    cnt_true = cnts[0]
    cnt_false = cnts[1]
    l_false = cnts[2]
    
print (cnt_true, cnt_false)
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['DM']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')



5 0
0.2 1.0 0.3333333333333333
(5, 0, 0.2, 1.0, 0.3333333333333333)
[]
[]
0.2 1.0 0.3333333333333333


51

In [518]:
#HCp20
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/HCp20/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['HC-p20']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


0.4 0.9090909090909091 0.5555555555555556
(10, 1, 0.4, 0.9090909090909091, 0.5555555555555556)
['GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A']
[0.27673649393605293]
0.4 0.9090909090909091 0.5555555555555556


71

In [519]:
#HCp200
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/HCp200/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            s = line.strip().split('\t')
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            allele = s[9].split(':')[1]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            ref = float(allele.split(',')[0])
            alt = float(allele.split(',')[1])
            VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['HC-p200']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')




0.4 0.8333333333333334 0.5405405405405405
(10, 2, 0.4, 0.8333333333333334, 0.5405405405405405)
['GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
[0.27673649393605293, 0.08284023668639054]
0.4 0.8333333333333334 0.5405405405405405


72

In [520]:
#MH
import glob
fl = glob.glob('/data/project/MRS/BioValidation/PlosGen/output/SS/MH/*.tsv')
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    sample = str(file).split('/')[-3]
    for line in f:
        sample = str(file).split('/')[-1]
        sample = sample.split('.')[0]
        s = line.strip().split('\t')
        chrom = s[0]
        pos = s[1]
        ref = s[6]
        alt = s[8]
        VAF = float(s[9])/int(s[7])
        info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
        cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
        cnt_true = cnts[0]
        cnt_false = cnts[1]
        l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)
        
l_result_tmp = ['MH']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp )) + '\t'+ DATA + '\t' + VAF_range + '\n')



0.08 1.0 0.14814814814814814
(2, 0, 0.08, 1.0, 0.14814814814814814)
[]
[]
0.08 1.0 0.14814814814814814


53

## Joint

In [521]:
#MT2P
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/MT2P/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            print (s[8])
            VAF = s[9].split(':')[2]
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['MT2P']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
GT:AD:AF:DP:F1R2:F2R1:FAD:SB
0.76 0.9047619047619048 0.8260869565217391
(19, 2, 0.76, 0.9047619047619048, 0.8260869565217391)
['GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
['9.500e-03', '2.899e-03']
0.76 0.9047619047619048 0.8260869565217391


70

In [522]:
def STK(line):
        dic_idx = {'A': 0, 'C': 1, 'G':2, 'T':3}
        if line[0] != '#':
               s = line.strip().split('\t')
               format = s[-1]
               ref = s[3]
               alt = s[4]
               l_format = format.split(':')[4:]
               dp_cnt = int(format.split(':')[0])
               alt_cnt= int(l_format[dic_idx[alt]].split(',')[0])
               vaf = float(alt_cnt)/dp_cnt
               return vaf

In [523]:
#STK
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/STK/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            print (s[8])
            VAF = STK(line)
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['STK']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
0.68 0.85 0.7555555555555555
(17, 3, 0.68, 0.85, 0.7555555555555555)
['GBM-P-0026-Tumor-F-Rep2:chr19:8371354:G:A', 'GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
[0.06031746031746032, 0.2763440860215054, 0.08357348703170028]
0.68 0.85 0.7555555555555555


55

In [524]:
#M2S2MH
import glob
fl = glob.glob("/data/project/MRS/BioValidation/PlosGen/output/SS/M2S2MH/*tsv")
l_true, l_false,l_false_VAF, cnt_true, cnt_false = [], [], [],0,0
for file in fl:
    f = open(file, 'r')
    for line in f:
        if line[:2] == '##':
            pass
        elif line[0] == '#':
            sample = line.strip().split('\t')[-1]
        else:
            sample = str(file).split('/')[-1]
            sample = sample.split('.')[0]
            s = line.strip().split('\t')
            chrom = s[0]
            pos = s[1]
            ref = s[3]
            alt = s[4]
            print (s[8])
            VAF = STK(line)
            info = sample + ':' + chrom + ':' + pos + ':' + ref + ':' + alt
            #ref = float(allele.split(',')[0])
            #alt = float(allele.split(',')[0])
            #VAF = alt/(ref+alt)
            cnts = Performance_gather(info,VAF, cnt_true, cnt_false,l_true, l_false, l_false_VAF)
            cnt_true = cnts[0]
            cnt_false = cnts[1]
            l_false = cnts[2]
        
print (Performance_check(cnt_true, cnt_false))
print (l_false)
print (l_false_VAF)

l_result_tmp = ['M2S2MH']
l_result_tmp.extend(list(Performance_check(cnt_true, cnt_false)))
out.write('\t'.join(map(str,l_result_tmp ))+ '\t'+ DATA + '\t' + VAF_range + '\n')


DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
DP:FDP:SDP:SUBDP:AU:CU:GU:TU
0.48 0.8571428571428571 0.6153846153846154
(12, 2, 0.48, 0.8571428571428571, 0.6153846153846154)
['GBM-P-0187-Tumor-M-Rep1:chr20:17456366:G:A', 'GBM-P-0881-Tumor-M-Rep1:chr1:110340663:A:T']
[0.2763440860215054, 0.08357348703170028]
0.48 0.8571428571428571 0.6153846153846154


72